In [3]:
import csv
import os
import numpy as np
from qsd.electromagnetics import cpw
from qsd.data_processing import setparams

# Define geometry of the superconductor
setp = setparams.SetParams()
params = setp.set_params("cpw_parameters.txt")

w = params["w"]
t = params["t"]
l = params["l"]
pen = params["pen"]
omega = params["omega"]
Z = params["Z"]

# Define the 'mesh'
x = np.linspace(-w, w, int(1e04))

# Instantiate Special CPW object
cpw = cpw.CPW(x,l,w,t,pen,Z,omega)

Js = cpw.J() #s Current density - not normalised
Jnorm = cpw.normalize_J() # Normalise
I = cpw.current(norm='no') # Find the current

# Generate a parameter list for COMSOL modelling
paramlist = setp.param_list(x,I,Jnorm,'paramlist.txt') # Generate COMSOL parameter list

currentDensityFile = str(os.getcwd() + "/current_density.csv")
np.savetxt(currentDensityFile, np.column_stack((x,Jnorm)), delimiter=",")

currentFile = str(os.getcwd() + "/current.csv")
np.savetxt(currentFile, np.column_stack((x,I)), delimiter=",")

In [2]:
import os
import numpy as np
from qsd.data_processing import readcomsol,postproc,setparams

# Read data from downloads
file_dbx = os.getcwd() + '/downloads/exports/Bx_fullData.csv'
file_dby = os.getcwd() + '/downloads/exports/By_fullData.csv'

rdx = readcomsol.ReadComsol(file_dbx)
rdy = readcomsol.ReadComsol(file_dby)

# Read csv file, and get x,y annd dbx/dby data for each
# blocked point in space
bx_x,bx_y,bx_z = rdx.read_full_data()
by_x,by_y,by_z = rdy.read_full_data()

dbx = np.asarray(bx_z).astype(np.float)
dby = np.asarray(by_z).astype(np.float)

# Define geometry of the superconductor
setp = setparams.SetParams()
params = setp.set_params("cpw_parameters.txt")

w = params["w"]
t = params["t"]
l = params["l"]
pen = params["pen"]
omega = params["omega"]
Z = params["Z"]

# Postprocess data
post = postproc.PostProc(w,t,l,pen,omega,Z)

# Single spin coupling for each point on mesh grid
g = post.coupling(dbx,dby,theta=0)
hist, edges = post.spin_density(bx_x,bx_y,g) # density

In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [10]:
from qsd.data_processing import readcomsol,postproc
import numpy as np
import os

#read in 1d data from comsol for plotting
bx = readcomsol.ReadComsol(os.getcwd() + '/downloads/exports/Bx.csv')
by = readcomsol.ReadComsol(os.getcwd() + '/downloads/exports/By.csv')
bn = readcomsol.ReadComsol(os.getcwd() + '/downloads/exports/normB.csv')

xx,Bx = bx.read_1D_comsol_data()
xy,By = by.read_1D_comsol_data()
xn,Bn = bn.read_1D_comsol_data()

# Define geometry of the superconductor
setp = setparams.SetParams()
params = setp.set_params("cpw_parameters.txt")

w = params["w"]
t = params["t"]
l = params["l"]
pen = params["pen"]
omega = params["omega"]
Z = params["Z"]

#calcualte single spin couplinng coefficient
pp = postproc.PostProc(w,t,l,pen,omega,Z)
g = pp.cut_line_single_spin_coupling(Bx,By)

rho = pp.cut_line_spin_density(g)
rho = rho / sum(rho)

IndexError: list index out of range

In [11]:
#!/usr/bin/env python
from qsd.data_processing import readcomsol,postproc
import numpy as np
import os

#read in 1d data from comsol for plotting
bx = readcomsol.ReadComsol(os.getcwd() + 'downloads/exports/Bx.csv')
by = readcomsol.ReadComsol(os.getcwd() + 'downloads/exports/By.csv')
bn = readcomsol.ReadComsol(os.getcwd() + 'downloads/exports/normB.csv')

xx,Bx = bx.read_1D_comsol_data()
xy,By = by.read_1D_comsol_data()
xn,Bn = bn.read_1D_comsol_data()

# Define geometry of the superconductor
setp = setparams.SetParams()
params = setp.set_params("cpw_parameters.txt")

w = params["w"]
t = params["t"]
l = params["l"]
pen = params["pen"]
omega = params["omega"]
Z = params["Z"]

lambda_c = 6e-03 # Will work out properly, but just testing for now
epsilon_r = 11.9
n = np.sqrt(epsilon_r) / sp.c # Dielectric constant
Q = 20000 # Will get this data from CST
F = pp.purcell_factor(lambda_c,n,Q)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/garethjones/PythonScripts/qsd/qsd/testsdownloads/exports/Bx.csv'